In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import mixture

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm, linear_model, datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#read in 2015-2022 datasets, add year column, add rank to years that don't have rank, clean column names 
df_15 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2015.csv')
df_15["Year"] = 2015
df_15.rename(columns = {'Family':'Social support'}, inplace = True)

df_16 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2016.csv')
df_16["Year"] = 2016
df_16.rename(columns = {'Family':'Social support'}, inplace = True)

df_17 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2017.csv')
df_17["Year"] = 2017
df_17. rename(columns = {'Happiness.Rank':'Happiness Rank', 'Happiness.Score':'Happiness Score',
                         'Economy..GDP.per.Capita.':'Economy (GDP per Capita)',
                         'Health..Life.Expectancy.':'Health (Life Expectancy)',
                         'Trust..Government.Corruption.':'Trust (Government Corruption)',
                        'Dystopia.Residual':'Dystopia Residual',
                        'Family':'Social support'}, inplace = True)

df_18 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2018.csv')
df_18["Year"] = 2018
df_18. rename(columns = {'Overall rank':'Happiness Rank', 'Score':'Happiness Score',
                         'Country or region':'Country',
                         'GDP per capita':'Economy (GDP per Capita)',
                         'Healthy life expectancy':'Health (Life Expectancy)',
                         'Freedom to make life choices':'Freedom',
                        'Perceptions of corruption':'Trust (Government Corruption)'}, inplace = True)

df_19 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2019.csv')
df_19["Year"] = 2019
df_19. rename(columns = {'Overall rank':'Happiness Rank', 'Score':'Happiness Score',
                         'Country or region':'Country',
                         'GDP per capita':'Economy (GDP per Capita)',
                         'Healthy life expectancy':'Health (Life Expectancy)',
                         'Freedom to make life choices':'Freedom',
                        'Perceptions of corruption':'Trust (Government Corruption)'}, inplace = True)

df_20 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2020.csv')
df_20["Year"] = 2020
df_20.rename(columns = {'Ladder score':'Happiness Score',
                         'Country name':'Country',
                         'Logged GDP per capita':'Economy (GDP per Capita)',
                         'Healthy life expectancy':'Health (Life Expectancy)',
                         'Freedom to make life choices':'Freedom',
                        'Perceptions of corruption':'Trust (Government Corruption)',
                       'Dystopia + residual':'Dystopia Residual'}, inplace = True)
df_20['Happiness Rank'] = df_20['Happiness Score'].rank(ascending=False)

df_21 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2021.csv')
df_21["Year"] = 2021
df_21.rename(columns = {'Ladder score':'Happiness Score',
                         'Country name':'Country',
                         'Logged GDP per capita':'Economy (GDP per Capita)',
                         'Healthy life expectancy':'Health (Life Expectancy)',
                         'Freedom to make life choices':'Freedom',
                        'Perceptions of corruption':'Trust (Government Corruption)',
                       'Dystopia + residual':'Dystopia Residual'}, inplace = True)
df_21['Happiness Rank'] = df_21['Happiness Score'].rank(ascending=False)


df_22 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2022.csv')
df_22["Year"] = 2022

In [3]:
#merge datasets
df = pd.concat([df_15, df_16, df_17, df_18, df_19, df_20, df_21])

list(df.columns) #will have to clean up column names so that they can merge properly
df = df.iloc[:, 0:13]
df = df.drop(['Region','Standard Error'], 1)

In [25]:
#rescale happiness scores
X = df["Happiness Score"].values.reshape(-1, 1)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X) # do scaling
df["Happiness Score"] = X_scaled

In [28]:
df["Score Category"] = pd.cut(df["Happiness Score"], bins=[0, .33, .67, 1.0], right=False, labels=['low', 'med', 'high'])

In [29]:
df

,Country,Happiness Rank,Happiness Score,Economy (GDP per Capita),Social support,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual,Year,Score Category
0,Switzerland,1.0,0.952059,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,2015,high
1,Iceland,2.0,0.947171,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201,2015,high
2,Denmark,3.0,0.940778,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,2015,high
3,Norway,4.0,0.939838,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,2015,high
4,Canada,5.0,0.921978,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,2015,high
...,...,...,...,...,...,...,...,...,...,...,...,...
144,Lesotho,145.0,0.185937,7.92600,0.78700,48.70000,0.71500,0.91500,-0.13100,1.80000,2021,low
145,Botswana,146.0,0.177477,9.78200,0.78400,59.26900,0.82400,0.80100,-0.24600,0.64800,2021,low
146,Rwanda,147.0,0.167701,7.67600,0.55200,61.40000,0.89700,0.16700,0.06100,1.09500,2021,low
147,Zimbabwe,148.0,0.116939,7.94300,0.75000,56.20100,0.67700,0.82100,-0.04700,1.20500,2021,low
